In [1]:
import pandas as pd
import numpy as np
import rewriter

In [2]:
df1=pd.read_csv('./tests/datasets/ibtesama__getting-started-with-a-movie-recommendation-system__tmdb_5000_credits.csv')
df2=pd.read_csv('./tests/datasets/ibtesama__getting-started-with-a-movie-recommendation-system__tmdb_5000_movies.csv')

# -- STEFANOS -- Replicate Data

In [3]:
factor = 400
df1 = pd.concat([df1]*factor)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 499
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   movie_id  200000 non-null  int64 
 1   title     200000 non-null  object
 2   cast      200000 non-null  object
 3   crew      200000 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.6+ MB


In [4]:
factor = 80
df2 = pd.concat([df2]*factor)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                40000 non-null  int64  
 1   genres                40000 non-null  object 
 2   homepage              24640 non-null  object 
 3   id                    40000 non-null  int64  
 4   keywords              40000 non-null  object 
 5   original_language     40000 non-null  object 
 6   original_title        40000 non-null  object 
 7   overview              40000 non-null  object 
 8   popularity            40000 non-null  float64
 9   production_companies  40000 non-null  object 
 10  production_countries  40000 non-null  object 
 11  release_date          40000 non-null  object 
 12  revenue               40000 non-null  int64  
 13  runtime               40000 non-null  int64  
 14  spoken_languages      40000 non-null  object 
 15  status               

# -- STEFANOS -- Prelude

In [5]:
%%time
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

CPU times: user 1.6 s, sys: 866 ms, total: 2.46 s
Wall time: 2.47 s


In [6]:
C= df2['vote_average'].mean()

In [7]:
m= df2['vote_count'].quantile(0.9)

In [8]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

(1600000, 23)

# -- STEFANOS -- `apply()` with `axis=1` and only assignments and math

*Original time*: 14.9s

*Rewritten*: 13.4ms

## Speedup: 1146x

In [9]:
%%time
%%rewrite
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

--------- Final Source -----------
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return v / (v + m) * R + m / (m + v) * C
q_movies['score'] = weighted_rating(q_movies)

CPU times: user 42 ms, sys: 26.2 ms, total: 68.3 ms
Wall time: 13.4 ms
